In [1]:
import os
from PIL import Image
import pandas as pd
import numpy as np

# Put a Cyrillic folder in the same folder as this file
cyrillic_path = "./Cyrillic"
        
df = pd.DataFrame()

binarizer = np.vectorize(lambda i: 1 if i > 0 else 0)

for dirName, subdirList, fileNames in os.walk(cyrillic_path):
    for i, fileName in enumerate(fileNames):
        if fileName.endswith(".png"):
#             print "{n}. {dirName}/{fileName}".format(n=i, dirName=dirName, fileName=fileName)
            img = Image.open(os.path.join(dirName, fileName))
            img = img.resize((50, 50), Image.ANTIALIAS)
            bitmap = np.array(img.getdata())
            bitmap = bitmap[:,3] # only need the fourth item in the RGB (transparency)
            bitmap = binarizer(bitmap) # change bitmap to 1 or 0
            img.close()
            character = dirName.split("/")[-1]
            df = df.append(pd.DataFrame([[character, bitmap]], columns=['classification', 'imageData']), ignore_index=True)

df.head()

""


In [76]:
y_data = pd.Categorical(df["classification"]).codes
x_data = list(df["imageData"].values)

In [1]:
y_data

NameError: name 'y_data' is not defined

# Train SVM Classifier

In [77]:
from sklearn.model_selection import train_test_split
from sklearn import svm

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3)

In [78]:
from sklearn.multiclass import OneVsRestClassifier

clf = OneVsRestClassifier(svm.SVC(verbose=True))
clf.fit(x_train, y_train)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True),
          n_jobs=1)

In [79]:
clf.score(x_test, y_test)

0.5768733850129198

# Get Predictions

In [80]:
y_pred = clf.predict(x_test)

In [96]:
y_data.shape

(15480,)

# ROC

In [92]:
from sklearn.metrics import roc_curve

for i in enumerate(classifications):
    print i
    fpr, tpr, _thresholds = roc_curve(y_test[:,i], y_pred[:,i])
    plt.plot(fpr, tpr, 'b-')

plt.plot([[0,0],[0,1]], 'k--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show()

numpy.ndarray

# AUC

In [ ]:
from sklearn.metrics import auc

auc(fpr, tpr)

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()
(tn, fp, fn, tp)